#### 핵심내용
- 데이터 선택 및 필터링하기
- 열 생성 및 삭제하기
- 결측값이 있는 열 검색 및 수정하기
- 데이터 프레임의 인덱싱과 정렬하기
- join과 union을 사용해 데이터 프레임 결합하기
- 텍스트 형식과 파케이 형식으로 데이터 프레임 작성하기

In [1]:
# library
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np

In [2]:
dtypes = {
 'Date First Observed': str,
 'Days Parking In Effect    ': str,
 'Double Parking Violation': str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': str,
 'House Number': str,
 'Hydrant Violation': str,
 'Intersecting Street': str,
 'Issue Date': str,
 'Issuer Code': np.float32,
 'Issuer Command': str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': str,
 'Issuing Agency': str,
 'Law Section': np.float32,
 'Meter Number': str,
 'No Standing or Stopping Violation': str,
 'Plate ID': str,
 'Plate Type': str,
 'Registration State': str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': str,
 'Sub Division': str,
 'Summons Number': np.uint32,
 'Time First Observed': str,
 'To Hours In Effect': str,
 'Unregistered Vehicle?': str,
 'Vehicle Body Type': str,
 'Vehicle Color': str,
 'Vehicle Expiration Date': str,
 'Vehicle Make': str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': str,
 'Violation Description': str,
 'Violation In Front Of Or Opposite': str,
 'Violation Legal Code': str,
 'Violation Location': str,
 'Violation Post Code': str,
 'Violation Precinct': np.float32,
 'Violation Time': str
}

In [4]:
nyc_data_raw = dd.read_csv('./data/*.csv', dtype=dtypes, usecols=dtypes.keys())

#### 1. 인덱스 및 축 작업하기

In [5]:
# 데이터 프레임에서 단일 열 선택하기
with ProgressBar() : 
    display(nyc_data_raw['Plate ID'].head())

[########################################] | 100% Completed |  1.9s


0    GBB9093
1    62416MB
2    78755JZ
3    63009MA
4    91648MC
Name: Plate ID, dtype: object

In [6]:
# 데이터 프레임에서 여러 열 선택하기
with ProgressBar() :
    display(nyc_data_raw[['Plate ID', 'Registration State']].head())

[########################################] | 100% Completed |  1.8s


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


In [8]:
# 데이터 프레임에서 열 삭제하기
with ProgressBar() :
    display(nyc_data_raw.drop('Violation Code', axis=1).head())

[########################################] | 100% Completed |  1.9s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,GY,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,WH,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,WH,0,2010.0,-,0.0,NaN,NaN,NaN,NaN,NaN
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,BR,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN


In [11]:
# 데이터 프레임에서 여러 열 삭제하기
violationColumnNames = list(filter(
    lambda columnName : 'Violation' in columnName, nyc_data_raw.columns)
                           )
with ProgressBar() : 
    display(nyc_data_raw.drop(violationColumnNames, axis=1).head())

[########################################] | 100% Completed |  1.9s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,408.0,F1,BBBBBBB,ALL,ALL,GY,0,2013.0,-,0.0
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,408.0,C,BBBBBBB,ALL,ALL,WH,0,2012.0,-,0.0
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,408.0,F7,BBBBBBB,ALL,ALL,NaN,0,0.0,-,0.0
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,408.0,F1,BBBBBBB,ALL,ALL,WH,0,2010.0,-,0.0
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,408.0,E1,BBBBBBB,ALL,ALL,BR,0,2012.0,-,0.0


In [13]:
# 열이름 바꾸기
nyc_data_renamed = nyc_data_raw.rename(columns={'Plate ID' : 'License Plate'})
nyc_data_renamed

,Summons Number,License Plate,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=138,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,object,object,object,object,uint16,object,object,object,uint32,uint32,uint32,object,object,float32,float32,float32,object,object,object,object,object,object,object,object,object,object,float32,object,object,object,object,object,object,object,float32,object,float32,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
# 단일 행 선택하기
with ProgressBar() : 
    display(nyc_data_raw.loc[56].head(1))

[########################################] | 100% Completed |  1.7s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
56,1293090530,GES3519,NY,PAS,07/07/2013,40,SDN,HONDA,F,70630,...,BLACK,0,1997.0,-,0.0,NaN,NaN,NaN,NaN,NaN


In [15]:
# 인덱스로 행을 순차적으로 가져오기
with ProgressBar() :
    display(nyc_data_raw.loc[100:200].head(100))

[########################################] | 100% Completed |  1.7s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
100,1294727205,XBD7628,VA,PAS,08/04/2013,17,SUBN,JEEP,P,14510,...,GRY,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
101,1294727461,R613159,IL,PAS,07/17/2013,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
102,1294727473,6TCX735,CA,PAS,07/18/2013,17,SDN,MAZDA,P,14510,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
103,1294727497,ZWZ43K,NJ,PAS,08/10/2013,17,SUBN,LINCO,P,14510,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
104,1295357240,T624858C,NY,PAS,07/22/2013,21,SUBN,TOYOT,X,28790,...,SILVE,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1307953700,481CKW,NJ,PAS,07/27/2013,24,SUBN,NISSA,K,0,...,PURPL,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
196,1307953724,W122KP,NY,PAS,07/28/2013,20,SDN,TOYOT,K,0,...,MAROO,0,1998.0,-,0.0,NaN,NaN,NaN,NaN,NaN
197,1307953761,GAW6458,NY,PAS,08/17/2013,20,SDN,TOYOT,K,48502,...,BLACK,0,2001.0,-,0.0,NaN,NaN,NaN,NaN,NaN
198,1307953852,GFP6251,NY,PAS,08/24/2013,27,SDN,NISSA,K,0,...,G/Y,0,2006.0,-,0.0,NaN,NaN,NaN,NaN,NaN


In [16]:
# 대스크와 팬더스를 사용해 행 슬라이스 필터링하기
with ProgressBar() :
    some_rows = nyc_data_raw.loc[100:200].head(100)
    
some_rows.drop(range(100, 200, 2))

[########################################] | 100% Completed |  1.7s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
101,1294727461,R613159,IL,PAS,07/17/2013,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
103,1294727497,ZWZ43K,NJ,PAS,08/10/2013,17,SUBN,LINCO,P,14510,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
105,1295546486,42909JM,NY,COM,07/17/2013,19,DELV,MERCU,P,58590,...,BROWN,0,1996.0,-,0.0,NaN,NaN,NaN,NaN,NaN
107,1302446484,452WW4,MA,PAS,06/24/2013,46,SDN,CHEVR,C,75530,...,YELLO,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
109,1302453774,GEX5296,NY,PAS,07/26/2013,14,SDN,ME/BE,C,36420,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
111,1302453798,867ZFX,CT,PAS,07/15/2013,17,SDN,ACURA,C,0,...,WHITE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
113,1302456593,GEZ1408,NY,PAS,07/15/2013,17,SUBN,HYUND,X,77730,...,BLK,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
115,1302456738,UCSLIM,NY,PAS,08/07/2013,17,SUBN,FORD,X,77730,...,BLK,0,2003.0,-,0.0,NaN,NaN,NaN,NaN,NaN
117,1302466203,HPM2636,PA,PAS,08/08/2013,40,SUBN,CADIL,X,24245,...,SILVR,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
119,1302466227,TWODAMAX,NY,SRF,08/05/2013,17,SDN,HYUND,X,53630,...,GRAY,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN


#### 2. 결측값 다루기

In [17]:
# 열별로 결측값 백분율 계산하기
missing_values = nyc_data_raw.isnull().sum()

with ProgressBar() : 
    percent_missing = ((missing_values / nyc_data_raw.index.size) * 100).compute()

percent_missing

[########################################] | 100% Completed |  6min  9.5s


Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

In [18]:
# 결측값이 50% 이상인 열 삭제
columns_to_drop = list(percent_missing[percent_missing >= 50].index)
nyc_data_clean_stage1 = nyc_data_raw.drop(columns_to_drop, axis=1)

In [19]:
# 결측값 대체하기
# 가정 : 누락된 값이 데이터셋에서 가장 일반적인 색상일 것.
with ProgressBar() :
    count_of_vehicle_colors = nyc_data_clean_stage1['Vehicle Color'].value_counts().compute()

most_common_color = count_of_vehicle_colors.sort_values(ascending=False).index[0]

nyc_data_clean_stage2 = nyc_data_clean_stage1.fillna({'Vehicle Color' : most_common_color})

[########################################] | 100% Completed |  3min 32.5s


In [22]:
with ProgressBar() :
    print(len(nyc_data_raw))

[########################################] | 100% Completed |  3min 17.2s
42339438


In [23]:
count_of_vehicle_colors

GY       6280314
WH       6074770
WHITE    5624960
BK       5121030
BLACK    2758479
          ...   
I.MG           1
I.MJE          1
I.NC           1
I.Y.A          1
white          1
Name: Vehicle Color, Length: 5744, dtype: int64

In [25]:
# 결측값이 있는 열 삭제하기
rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop)

In [27]:
rows_to_drop

['Plate ID',
 'Vehicle Body Type',
 'Vehicle Make',
 'Vehicle Expiration Date',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Violation Time',
 'Street Name',
 'Date First Observed',
 'Law Section',
 'Sub Division',
 'Vehicle Color',
 'Vehicle Year',
 'Feet From Curb']

In [28]:
# 남은 열들의 데이터 타입 찾기
remaining_columns_to_clean = list(
    percent_missing[(percent_missing >=5) & percent_missing <50].index
)
nyc_data_raw.dtypes[remaining_columns_to_clean]

Summons Number                        uint32
Plate ID                              object
Registration State                    object
Plate Type                            object
Issue Date                            object
Violation Code                        uint16
Vehicle Body Type                     object
Vehicle Make                          object
Issuing Agency                        object
Street Code1                          uint32
Street Code2                          uint32
Street Code3                          uint32
Vehicle Expiration Date               object
Violation Location                    object
Violation Precinct                   float32
Issuer Precinct                      float32
Issuer Code                          float32
Issuer Command                        object
Issuer Squad                          object
Violation Time                        object
Time First Observed                   object
Violation County                      object
Violation 

In [30]:
# fillna를 위한 대체값 딕셔너리 만들기
unknown_default_dict = dict(map(
    lambda columnName: (columnName, 'Unknown'), remaining_columns_to_clean
    )
)

In [32]:
# 기본값으로 데이터 프레임 채우기
nyc_data_clean_stage4 = nyc_data_clean_stage3.fillna(unknown_default_dict)

In [33]:
nyc_data_clean_stage4

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Date First Observed,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
npartitions=138,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,object,object,object,object,uint16,object,object,object,uint32,uint32,uint32,object,object,float32,float32,float32,object,object,object,object,object,object,object,object,float32,object,object,object,object,object,float32,float32,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [34]:
# 채우기/삭제 작업이 잘 됐는지 확인하기
with ProgressBar() :
    print(nyc_data_clean_stage4.isnull().sum().compute())

nyc_data_clean_stage4.persist()

[########################################] | 100% Completed |  9min 48.4s
Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                  

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Date First Observed,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
npartitions=138,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,object,object,object,object,uint16,object,object,object,uint32,uint32,uint32,object,object,float32,float32,float32,object,object,object,object,object,object,object,object,float32,object,object,object,object,object,float32,float32,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


#### 3. 데이터 기록하기

In [35]:
# Plate Type 열의 값 횟수 구하기
with ProgressBar() :
    license_plate_types = nyc_data_clean_stage4['Plate Type'].value_counts().compute()

license_plate_types

[########################################] | 100% Completed |  8min  6.5s


PAS    30452502
COM     7966914
OMT     1389341
SRF      394656
OMS      368952
         ...   
HOU           4
JWV           3
LOC           3
HIF           2
SNO           2
Name: Plate Type, Length: 90, dtype: int64

In [38]:
# Plate Type 열 기록하기
condition = nyc_data_clean_stage4['Plate Type'].isin(['PAS', 'COM'])
plate_type_masked = nyc_data_clean_stage4['Plate Type'].where(condition, 'Other')
nyc_data_recode_stage1 = nyc_data_clean_stage4.drop('Plate Type', axis=1)
nyc_data_recode_stage2 = nyc_data_recode_stage1.assign(PlateType=plate_type_masked)
nyc_data_recode_stage3 = nyc_data_recode_stage2.rename(columns={'PlateType':'Plate Type'})

In [39]:
# 리코딩 후 값 횟수 보기
with ProgressBar() :
    display(nyc_data_recode_stage3['Plate Type'].value_counts().compute())

[########################################] | 100% Completed |  8min 46.4s


PAS      30452502
COM       7966914
Other     3418586
Name: Plate Type, dtype: int64

- where : 전달된 조건이 False일 때, 값을 대체
  - 많은 고유값들 중 조금만 유지하려는 경우
- mask : 전달된 조건이 True일 때, 값을 대체
  - 고유값들 중 몇 개만 삭제하고자 하는 경우

In [40]:
# mask를 사용해 'Other' 범주에 고유한 색상 배치하기
single_color = list(count_of_vehicle_colors[count_of_vehicle_colors == 1].index)
condition = nyc_data_clean_stage4['Vehicle Color'].isin(single_color)
vehicle_color_masked = nyc_data_clean_stage4['Vehicle Color'].mask(condition, 'Other')
nyc_data_recode_stage4 = nyc_data_recode_stage3.drop('Vehicle Color', axis=1)
nyc_data_recode_stage5 = nyc_data_recode_stage4.assign(VehicleColor=vehicle_color_masked)
nyc_data_recode_stage6 = nyc_data_recode_stage5.rename(columns={'VehicleColor':'Vehicle Color'})

#### 3. 요소별 연산

In [42]:
# 발행 날짜 열 분석하기
from datetime import datetime

issue_date_parsed = nyc_data_recode_stage6['Issue Date'].apply(
    lambda x : datetime.strptime(x, '%m/%d/%Y'), meta=datetime 
    # meta=datetime : Dask의 경우 타입을 명시적으로 지정해주는게 좋다.
)
nyc_data_derived_stage1 = nyc_data_recode_stage6.drop('Issue Date', axis=1)
nyc_data_derived_stage2 = nyc_data_derived_stage1.assign(IssueDate=issue_date_parsed)
nyc_data_derived_stage3 = nyc_data_derived_stage2.rename(columns={'IssueDate':'Issue Date'})

/Users/cbm/workspace/dask/venv/lib/python3.9/site-packages/dask/dataframe/core.py:6240: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [43]:
# 날짜 파싱 결과 조사하기
with ProgressBar() :
    display(nyc_data_derived_stage3['Issue Date'].head())

[########################################] | 100% Completed |  6.8s


0   2013-08-04
1   2013-08-04
2   2013-08-05
3   2013-08-05
4   2013-08-08
Name: Issue Date, dtype: datetime64[ns]

In [44]:
# 월과 연도 추출하기
issue_date_month_year = nyc_data_derived_stage3['Issue Date'].apply(
    lambda dt : dt.strftime('%Y%m'), meta=int
)
nyc_data_derived_stage4 = nyc_data_derived_stage3.assign(IssueMonthYear=issue_date_month_year)
nyc_data_derived_stage5 = nyc_data_derived_stage4.rename(
    columns={'IssueMonthYear':'Citation Issued Month Year'}
)

/Users/cbm/workspace/dask/venv/lib/python3.9/site-packages/dask/dataframe/core.py:6240: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [45]:
# 새로운 파생 열 조사하기
with ProgressBar() :
    display(nyc_data_derived_stage5['Citation Issued Month Year'].head())

[########################################] | 100% Completed |  8.7s


0    201308
1    201308
2    201308
3    201308
4    201308
Name: Citation Issued Month Year, dtype: object

#### 4. 데이터 프레임의 필터링과 재색인

In [46]:
# 10월에 발생한 모든 주차 위반 찾기
months = ['201310', '201410', '201510', '201610', '201710']
condition = nyc_data_derived_stage5['Citation Issued Month Year'].isin(months)
october_citations = nyc_data_derived_stage5[condition]

with ProgressBar() :
    display(october_citations.head())

[########################################] | 100% Completed |  8.7s


,Summons Number,Plate ID,Registration State,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Issue Date,Citation Issued Month Year
1609,1340313923,GEK8055,NY,40,SUBN,HONDA,P,79630,40404,40404,...,ALL,ALL,2013.0,5.0,Unknown,Unknown,PAS,BROWN,2013-10-23,201310
23367,1351679867,XE726658,DE,20,P-U,DODGE,P,90980,0,0,...,ALL,ALL,2002.0,0.0,Unknown,Unknown,PAS,RED,2013-10-21,201310
24172,1351805253,42067JM,NY,14,DELV,FRUEH,P,25630,13610,24985,...,ALL,ALL,1999.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-17,201310
32902,1355051060,76254JY,NY,46,DELV,FRUEH,P,68020,26760,66120,...,ALL,ALL,2007.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-10,201310
32903,1355051071,44125MC,NY,46,VAN,FORD,P,68020,26490,26520,...,ALL,ALL,2011.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-10,201310


In [47]:
# 2016년 4월 25일 이후의 모든 주차 위반 기록 찾기
bound_date = '2016-4-25'
condition = nyc_data_derived_stage5['Issue Date'] > bound_date
citations_after_bound = nyc_data_derived_stage5[condition]

with ProgressBar() :
    display(citations_after_bound.head())

[########################################] | 100% Completed |  8.6s


,Summons Number,Plate ID,Registration State,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Issue Date,Citation Issued Month Year
3741,1346495701,GCJ8613,NY,20,SDN,DODGE,X,10210,19210,19250,...,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,2017-08-04,201708
3748,1346551819,GHJ2373,NY,20,SUBN,FORD,X,0,0,0,...,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,2019-07-22,201907
6905,1348077426,66811MB,NY,78,IR,FRUEH,P,25680,46320,8120,...,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHT,2053-08-02,205308
12967,1353305650,91609MC,NY,46,VAN,FORD,T,24890,18670,18690,...,ALL,ALL,2012.0,0.0,Unknown,Unknown,COM,BR,2016-08-07,201608
17144,1354617988,54015JV,NY,78,VAN,FORD,P,38590,50150,52290,...,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHITE,2032-07-28,203207


In [49]:
# 데이터 프레임에서 인덱스 설정하기
with ProgressBar() :
    condition = (nyc_data_derived_stage5['Issue Date'] > '2014-01-01') & \
                (nyc_data_derived_stage5['Issue Date'] < '2017-12-31')
    # 데이터를 필터링하여 2014-01-01과 2017-12-31 사이에 발행된 티켓만 유지
    nyc_data_filtered = nyc_data_derived_stage5[condition]
    nyc_data_new_index = nyc_data_filtered.set_index('Citation Issued Month Year')
    # 데이터 프레임의 인덱스를 월/년 열로 설정

[########################################] | 100% Completed |  1hr  6min 11.4s


/Users/cbm/workspace/dask/venv/lib/python3.9/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


ValueError: values cannot be losslessly cast to int64

In [ ]:
# 월/연도별 데이터 재분할
years = ['2014', '2015', '2016', '2017']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
divisions = [year + month for year in years for month in months]

with ProgressBar() :
    nyc_data_new_index.repartition(divisions=divisions) \
    .to_parquet('nyc_data_date_index', compression='snappy')

nyc_data_new_index = dd.read_parquet('nyc_data_date_index')

#### 5. 데이터 조인하기

In [53]:
import pandas as pd

nyc_temps = pd.read_csv('./data/nyc-temp-data.csv')
nyc_temps_indexed = nyc_temps.set_index(nyc_temps.monthYear.astype(str))

nyc_datg_with_temps = nyc_data_new_index.join(nyc_temps_indexed, how='inner')

with ProgressBar() : 
    display(nyc_data_with_temps.head(15))

NameError: name 'nyc_data_new_index' is not defined